In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

In [5]:
from dotenv import load_dotenv
import json, requests

In [8]:
usa_offices = list(db.companies_usa.find({'offices.demographics':{'$exists':True}}, {'offices.latitude':1, 'offices.longitude':1}))

In [11]:
latitude = usa_offices[0]['offices']['latitude']
longitude = usa_offices[0]['offices']['longitude']

## Nightlife spots at 10km radius (Foursquare)

In [96]:
load_dotenv()
foursq_api_id = os.getenv('foursquare_id')
foursq_api_k = os.getenv('foursquare_key')

In [167]:
# Foursquare categories: https://developer.foursquare.com/docs/build-with-foursquare/categories/

url = 'https://api.foursquare.com/v2/venues/search'

params = dict(
    client_id = f"{foursq_api_id}",
    client_secret = f"{foursq_api_k}",
    ll = f"{latitude},{longitude}",
    radius = '10000',
    categoryId = '4d4b7105d754a06376d81259',
    v = 20201115)

In [168]:
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [170]:
len(data['response']['venues'])

11

## Schools - Primary-schools in a 5km radius (Foursquare/Google)

In [209]:
url = 'https://api.foursquare.com/v2/venues/search'

params = dict(
    client_id = f"{foursq_api_id}",
    client_secret = f"{foursq_api_k}",
    ll = f"{latitude},{longitude}",
    radius = '4000',
    query = 'school',
    categoryId = '4f4533814b9074f6e4fb0107,52e81612bcbc57f1066b7a45,4f4533804b9074f6e4fb0105',
    v = 20201115)

In [210]:
resp = requests.get(url=url, params=params)
data_fs = json.loads(resp.text)

In [213]:
for e in data_fs['response']['venues']:
    print(e['name'])

Stratford School - Pleasanton
Frederiksen Elementary School
Fairlands Elementary School
Donlon Elementary School
James Dougherty Elementary School
Lydiksen Elementary School
Walnut Grove Elementary School
Primrose School of Pleasanton
Learn And Play Montessori School
Nielsen Elementary School
Valley Christian Elementary School
Henry P. Mohr Elementary School
H. W. Kolb Elementary School
Alisal Elementary School
Murray Elementary School
Kolb Elementary School


In [101]:
load_dotenv()
goog_cloud_k = os.getenv('google_cloud')

In [202]:
# Google categories: https://developers.google.com/places/web-service/supported_types

url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = dict(
    location = f"{latitude},{longitude}",
    radius = '4000',
    keyword = 'school',
    type = 'primary_school',
    key = f"{goog_cloud_k}")

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [205]:
for e in data['results']:
    print(e['name'])

Dublin Elementary School
Fairlands Elementary School
Montessori School of Pleasanton
Murray Elementary School
James Dougherty Elementary School
Kolb Elementary School
Alisal Elementary School
Walnut Grove Elementary School
Valley View Elementary School
John Green Elementary School
Donlon Elementary
Mohr Elementary School
Stratford School - Pleasanton
Nielsen Elementary School
Country Club Elementary School
Fairlands Elementary


## Tech startups and Design companies

In [62]:
usa_offices_all = list(db.companies_usa.find({}, {'category_code':1}))
categories = set([e['category_code'] for e in usa_offices_all])
print(categories)

{'software', 'nanotech', 'search', 'transportation', 'education', 'enterprise', 'health', 'games_video', 'fashion', 'cleantech', 'manufacturing', 'real_estate', 'security', 'finance', 'legal', 'design', 'advertising', 'news', 'network_hosting', 'hospitality', 'social', 'ecommerce', 'analytics', 'music', 'web', 'other', 'sports', 'medical', 'mobile', 'travel', 'public_relations', 'biotech', 'hardware', None, 'photo_video', 'consulting', 'nonprofit', 'messaging', 'automotive', 'semiconductor'}


In [235]:
usa_offices_all = list(db.companies_usa.find({'founded_year':{'$ne':None}}, {'founded_year':1}))
years = list(set([e['founded_year'] for e in usa_offices_all]))
years.sort()
print(years)

[1836, 1840, 1843, 1846, 1857, 1863, 1879, 1887, 1888, 1889, 1890, 1894, 1896, 1898, 1900, 1901, 1902, 1906, 1912, 1920, 1922, 1923, 1928, 1930, 1936, 1938, 1939, 1943, 1945, 1946, 1947, 1952, 1954, 1955, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1966, 1967, 1968, 1969, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]


In [214]:
tech_design = ['software', 'nanotech', 'games_video', 'cleantech', 'design', 'network_hosting', 'web', 'other', 'mobile', 'biotech']

In [236]:
tech_design_comp = list(db.companies_usa.find({'category_code':{'$in':tech_design}, 'founded_year':{'$gte':2010}}))

## Transport

In [249]:
load_dotenv()
lufthansa_id = os.getenv('lufthansa_id')
lufthansa_secret = os.getenv('lufthansa_secret')

In [239]:
print(latitude, longitude)

37.692934 -121.904945


In [271]:
request = 'https://api.lufthansa.com/v1/oauth/token'
header_auth = {
            'client_id' : f'{lufthansa_id}',
            'client_secret': f'{lufthansa_secret}', 
            'grant_type' : 'client_credentials'}      

r = requests.post(request, data = header_auth)
j = r.json()
token = j['access_token']
header = { 'Authorization': 'Bearer ' + str(token), 'Accept': 'application/json'}

In [272]:
resp = requests.get(f'https://api.lufthansa.com/v1/references/airports/nearest/{latitude},{longitude}?lang=en', headers=header)
data_air = json.loads(resp.text) 